In [84]:
import glob
import json
import os
from urllib.request import urlretrieve
from multiprocessing import Pool
from collections import defaultdict, namedtuple
from pprint import pprint
import tqdm

In [123]:
SUPPORTED_TYPES = ['photo', 'audio']
DUMP_DIR = 'dumps/nikkorobk/'

In [124]:
def photo_url_extractor(photo_at):
    return(photo_at['photo']['sizes'][-1]['url'])
def audio_url_extractor(audio_at):
    return(audio_at['audio']['url'].split('?')[0])


In [125]:
def extract_attachment_url(at):
    att = at['type']
    if att == 'photo':
        return photo_url_extractor(at)
    if att == 'audio':
        return audio_url_extractor(at)
    return ''

In [126]:
DTask = namedtuple('DTask',['url', 'type', 'user_id'])

def get_download_tasks(dump_dir):
    tasks = []
    files_list=glob.glob(os.path.join(dump_dir, '*.json'))
    for file in tqdm.tqdm_notebook(files_list):
        user_urls = defaultdict(list)
        user_id = os.path.splitext(os.path.basename(file))[0]
        conv = json.load(open(file))
        for msg in conv['items']:
            for at in msg['attachments']:
                att = at['type']
                if att in SUPPORTED_TYPES:
                    url = extract_attachment_url(at)
                    if url:
                        tasks.append(DTask(url, att, user_id))
    return tasks


In [127]:
def process_task(task):
    path_to_save =  os.path.join(DUMP_DIR, 'attachments', task.user_id, task.type)
    filename = task.url.split('/')[-1]
    full_name = os.path.join(path_to_save, filename)
    if not os.path.exists(path_to_save):
        os.makedirs(path_to_save)
    if not os.path.exists(full_name):
        urlretrieve(task.url, full_name)

In [128]:
tasks = get_download_tasks(DUMP_DIR)

In [135]:
def process_tasks(tasks):
    with Pool(processes=8) as p:
        with tqdm.tqdm_notebook(total = len(tasks)) as td:
            for r in p.imap_unordered(process_task, tasks):
                td.update()


In [136]:
process_tasks(audio_tasks)

Process ForkPoolWorker-68:
Process ForkPoolWorker-66:
Process ForkPoolWorker-67:
Process ForkPoolWorker-65:
Process ForkPoolWorker-69:
Process ForkPoolWorker-72:
Process ForkPoolWorker-71:
Process ForkPoolWorker-70:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/local/Cellar/python/3.7.1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/local/Cellar/python/3.7.1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/local/Cellar/python/3.7.1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/local

KeyboardInterrupt: 

In [133]:
audio_tasks = list(filter(lambda x: x.type == 'audio', tasks))

In [134]:
len(list(audio_tasks))

1914

In [152]:
audio_urls = list(map(itemgetter('url'), audio_tasks))

len(audio_urls), len(set(audio_urls))


TypeError: tuple indices must be integers or slices, not str

In [103]:
os.path.exists('dumps/nikkorobk/1300.json')

False

In [141]:
from operator import itemgetter